In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dsaa-6100-movie-review-sentiment-classification/test_data.csv
/kaggle/input/dsaa-6100-movie-review-sentiment-classification/movie_reviews/movie_reviews.csv


In [19]:
import pandas as pd

df_train = pd.read_csv('/kaggle/input/dsaa-6100-movie-review-sentiment-classification/movie_reviews/movie_reviews.csv')
df_test = pd.read_csv('/kaggle/input/dsaa-6100-movie-review-sentiment-classification/test_data.csv')

df_train.shape, df_test.shape

((40000, 2), (10000, 2))

In [40]:
import re

def clean_text(text):
    t = text.lower()
    t = re.sub(r'[^A-Za-z\s]', '', t)
    return t

def create_vocab(text:str, vocab:list=None, max_words=100):
    if vocab is None:
        print('intialize the vocab list')
        return
    text = clean_text(text)
    words = [word for word in text.split(' ') if word]
    final_words = []
    for word in words:
        if word not in vocab:
            final_words.append(word)
    
    vocab.extend(final_words)
    return vocab

def encode_text(text:str, vocab:list, min_len=10):
    txt_ids = []
    text = clean_text(text)
    words = [word for word in text.split(' ') if word]
    for word in words:  
        id = vocab.index(word)
        txt_ids.append(id)
    padding = abs(min_len - len(txt_ids))
    
    if len(txt_ids) < min_len:
        txt_ids = txt_ids + [0] * padding
    elif len(txt_ids) > min_len:
        txt_ids = txt_ids[:min_len]
    return txt_ids



In [23]:
import numpy as np
import re

max_len = 0


for text in df_train['text']:
    clean = clean_text(text)
    words = [word for word in clean.split(' ') if word]
    if len(words) > max_len:
        max_len = len(words)

print(max_len)

2450


In [ ]:
vocab=[]
for text in df_train['text']:
    vocab=create_vocab(text, vocab)
len(vocab)

In [32]:
import pickle 

with open('/kaggle/working/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

print(len(vocab))

159227


In [36]:
import pandas as pd

df_train = pd.read_csv('/kaggle/input/dsaa-6100-movie-review-sentiment-classification/movie_reviews/movie_reviews.csv')

In [43]:
from tqdm import tqdm
import numpy as np

def safe_encode_text(x, vocab, min_len=512):
    try:
        encoded = encode_text(x, vocab, min_len)
        # Normalize indices by dividing by vocabulary size
        normalized = np.array(encoded, dtype=np.float32) / len(vocab)
        return normalized
    except Exception as e:
        print(f"Error encoding text: {str(e)[:100]}")
        return np.zeros(min_len, dtype=np.float32)

# Using tqdm for progress tracking
tqdm.pandas()
df_train['txt_ids'] = df_train['text'].progress_map(lambda x: safe_encode_text(x, vocab))

# Convert the entire column to a list of numpy arrays if needed
df_train['txt_ids'] = df_train['txt_ids'].apply(lambda x: np.array(x, dtype=np.float32) if isinstance(x, str) else x)

# Verify the data type and values
print("Type of first element:", type(df_train['txt_ids'].iloc[0]))
print("Shape of first element:", df_train['txt_ids'].iloc[0].shape)
print("Data type of first element:", df_train['txt_ids'].iloc[0].dtype)
print("Min value:", df_train['txt_ids'].iloc[0].min())
print("Max value:", df_train['txt_ids'].iloc[0].max())

100%|██████████| 40000/40000 [09:43<00:00, 68.57it/s] 


Type of first element: <class 'numpy.ndarray'>
Shape of first element: (512,)
Data type of first element: float64
Min value: 0.0
Max value: 0.0010111350461919147


In [44]:
df_train.to_csv('df_train_encoded.csv', index=False)

In [45]:
df_train.head()

,text,label,txt_ids
0,If you havent seen this movie than you need to...,1,"[0.0, 6.280341901813135e-06, 1.256068380362627..."
1,but Cinderella gets my vote not only for the w...,0,"[0.0010174153880937278, 0.001023695729995541, ..."
2,This movie is pretty cheesy but I do give it c...,1,"[2.512136760725254e-05, 3.1401709509065676e-05..."
3,I have not seen a Van Damme flick for a while ...,1,"[0.00023237265036708598, 8.792478662538388e-05..."
4,This is a sleeper It defines Nicholas Cage The...,1,"[2.512136760725254e-05, 0.00011932649613444957..."


In [47]:
import numpy as np

X = np.stack(df_train['txt_ids'].values)
print(X.shape)
y = df_final['label'].values
print(y.shape)

(40000, 512)
(40000,)


In [48]:
df_final['label'].value_counts()

label
0    20049
1    19951
Name: count, dtype: int64

In [71]:
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split

# 1. Dataset Creation
def create_tf_dataset(X, y, batch_size=32):
    """Create TensorFlow dataset"""
    return tf.data.Dataset.from_tensor_slices(
        (tf.cast(X, tf.float32), tf.cast(y, tf.float32))
    ).shuffle(1000).batch(batch_size)

# 2. Learning Rate Callback
class PrintLearningRate(keras.callbacks.Callback):
    """Callback to print learning rate"""
    def on_batch_end(self, batch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        logs['lr'] = lr

# 3. Callbacks Creation
def create_callbacks(params):
    """Create training callbacks"""
    return [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=params['early_stopping_patience'],
            restore_best_weights=True,
            verbose=0
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=params['reduce_lr_factor'],
            patience=params['reduce_lr_patience'],
            min_lr=1e-6,
            verbose=0
        ),
        keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=0
        ),
        PrintLearningRate()
    ]

# 4. Model Creation
def create_model(input_shape, params, dense_only=False):
    """Create the model architecture"""
    input_layer = keras.layers.Input(shape=input_shape)
    x = input_layer
    
    if not dense_only:
        # GRU layers
        x = keras.layers.Reshape((1, -1))(x)
        
        for units in params['gru_units']:
            x = keras.layers.GRU(
                units, 
                return_sequences=True if units != params['gru_units'][-1] else False,
                dropout=params['dropout_rate'],
                recurrent_dropout=params['dropout_rate']
            )(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Dropout(params['dropout_rate'])(x)
    
    # Dense layers
    units = params['dense_units']
    for _ in range(3):
        if units < 5:
            units = 5
        x = keras.layers.Dense(
            units,
            activation=keras.activations.gelu,
            kernel_initializer=keras.initializers.he_normal,
            kernel_regularizer=keras.regularizers.l1_l2(params['l1_l2_reg']),
        )(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(rate=params['dropout_rate'])(x)
        units //= 2

    output_layer = keras.layers.Dense(1, activation=keras.activations.sigmoid)(x)
    
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=params['initial_lr'],
            clipnorm=1.0
        ),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy]
    )
    
    return model

# 5. Main Training Function
def setup_and_train(X, y, 
                    resume_training=False,
                    dense_only=False,
                    model_path='best_model.keras',
                    gru_units=[1000, 750, 550],
                    dense_units=520,
                    dropout_rate=0.2,
                    initial_lr=1e-3,
                    early_stopping_patience=3,
                    reduce_lr_patience=2,
                    reduce_lr_factor=0.2,
                    epochs=200,
                    batch_size=512*2,
                    l1_l2_reg=0.01):
    """
    Main function to setup and train the model
    Args:
        resume_training: If True, loads previous model and continues training
        dense_only: If True, uses only dense layers without GRU
        reduce_lr_factor: Factor by which learning rate is reduced (default 0.2)
        Other args: Model hyperparameters
    """
    
    # Create parameter dictionary
    params = {
        'gru_units': gru_units,
        'dense_units': dense_units,
        'dropout_rate': dropout_rate,
        'initial_lr': initial_lr,
        'early_stopping_patience': early_stopping_patience,
        'reduce_lr_patience': reduce_lr_patience,
        'reduce_lr_factor': reduce_lr_factor,
        'epochs': epochs,
        'batch_size': batch_size,
        'l1_l2_reg': l1_l2_reg
    }
    
    # Split data
    X_full, X_test, y_full, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.3, random_state=0)
    
    # Setup strategy
    strategy = tf.distribute.MirroredStrategy()
    print(f'Number of devices: {strategy.num_replicas_in_sync}')
    print(f"Data shapes: {X_train.shape}, {X_val.shape}, {X_test.shape}\n")
    
    # Create or load model
    with strategy.scope():
        if resume_training:
            try:
                model = keras.models.load_model(model_path)
                print("Resuming training from saved model")
                keras.backend.set_value(model.optimizer.learning_rate, initial_lr)
            except:
                print(f"No saved model found at {model_path}. Creating new model.")
                model = create_model((X_train.shape[1],), params, dense_only)
        else:
            print("Creating new model")
            model = create_model((X_train.shape[1],), params, dense_only)
    
    # Create datasets
    train_dataset = create_tf_dataset(X_train, y_train, params['batch_size'])
    val_dataset = create_tf_dataset(X_val, y_val, params['batch_size'])
    test_dataset = create_tf_dataset(X_test, y_test, 128)
    
    # Train model
    history = model.fit(
        train_dataset,
        epochs=params['epochs'],
        validation_data=val_dataset,
        callbacks=create_callbacks(params),
        verbose=1
    )
    
    # Evaluate model
    test_results = model.evaluate(test_dataset, verbose=0)
    print(f"\nTest Loss: {test_results[0]:.4f}")
    print(f"Test Accuracy: {test_results[1]:.4f}")
    
    return model, history, params


print('done')




done


In [70]:
# 1. Create new model
model, history, params = setup_and_train(X, y, 
                    resume_training=False,
                    dense_only=False,
                    model_path='best_dense_only_model.keras',
                    gru_units=[1000, 750, 550],
                    dense_units=300,
                    dropout_rate=0.2,
                    initial_lr=0.1,
                    early_stopping_patience=20,
                    reduce_lr_patience=5,
                    reduce_lr_factor=0.5,
                    epochs=200,
                    batch_size=512*2,
                    l1_l2_reg=0.1)

""""
# 2. Resume training
model, history, params = setup_and_train(
    X, y,
    resume_training=True,
    initial_lr=5e-4,
    epochs=100,
    batch_size=1024
)
"""

Number of devices: 2
Data shapes: (22400, 512), (9600, 512), (8000, 512)

Creating new model
Epoch 1/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - binary_accuracy: 0.5017 - loss: 818.9078 - lr: 0.1000 - val_binary_accuracy: 0.4944 - val_loss: 409.0956 - learning_rate: 0.1000
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.4964 - loss: 330.0677 - lr: 0.1000 - val_binary_accuracy: 0.4942 - val_loss: 258.3411 - learning_rate: 0.1000
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - binary_accuracy: 0.5001 - loss: 267.9983 - lr: 0.1000 - val_binary_accuracy: 0.4996 - val_loss: 267.1272 - learning_rate: 0.1000
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - binary_accuracy: 0.5066 - loss: 259.9080 - lr: 0.1000 - val_binary_accuracy: 0.5006 - val_loss: 274.4033 - learning_rate: 0.1000
Epoch 5/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - binary_accuracy: 0.5010 - loss: 261.7596 - lr: 0.1000 - val_binary_accuracy: 0.4944 - val_loss: 251.1309 - learning_rate

KeyboardInterrupt: 

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['binary_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_binary_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()